# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries, watermark our file, initialise our Spark Session Builder and initialise our environment with required configuration

In [2]:
%pip install watermark
%pip install Minio

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import from_json, col, to_json, struct
import watermark
from minio import Minio

%matplotlib inline
%load_ext watermark

In [4]:
%watermark -n -v -m -g -iv


Python implementation: CPython
Python version       : 3.7.5
IPython version      : 7.16.1

Compiler    : Clang 11.0.3 (clang-1103.0.32.62)
OS          : Darwin
Release     : 20.4.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Git hash: 94e587d3819dd44872e368d143e20e5d77d4571c

sys      : 3.7.5 (default, Jul  7 2020, 21:09:45) 
[Clang 11.0.3 (clang-1103.0.32.62)]
watermark: 2.1.0
json     : 2.0.9



In [5]:
sparkSessionBuilder = SparkSession\
    .builder\
    .appName("Customer Churn ingest Pipeline")

In [6]:
os.environ['PYSPARK_SUBMIT_ARGS'] = \
'--packages \
org.postgresql:postgresql:42.2.10,\
org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5,\
org.apache.kafka:kafka-clients:2.4.0,\
org.apache.spark:spark-streaming_2.11:2.4.5,\
org.apache.hadoop:hadoop-aws:2.7.3 \
--conf spark.jars.ivy=/tmp \
--conf spark.hadoop.fs.s3a.endpoint=http://172.30.226.86:9000 \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--master spark://' + os.environ['SPARK_CLUSTER'] + ':7077 pyspark-shell '



KeyError: 'SPARK_CLUSTER'

###  Connect to Spark Cluster provided by OpenShift Platform

In [ ]:
spark = sparkSessionBuilder.getOrCreate()
spark.sparkContext.setLogLevel("INFO")
print('Spark context started.')

###  Declare our input data sources, import and combine them

In [ ]:
dataFrame_Customer = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/Customer-Churn_P1.csv")
dataFrame_Customer.printSchema()

In [ ]:
# dataFrame_Products = spark.read\
#                 .options(delimeter=',', inferSchema='True', header='True') \
#                 .csv("s3a://rawdata/Customer-Churn_P2.csv")
# dataFrame_Products.printSchema()


In [ ]:
from pyspark.sql.types import *

srcKafkaBrokers = "odh-message-bus-kafka-bootstrap:9092"
srcKakaTopic = "data"


schema = StructType()\
    .add("customerID", IntegerType())\
    .add("PhoneService", StringType())\
    .add("MultipleLines", StringType())\
    .add("InternetService", StringType())\
    .add("OnlineSecurity", StringType())\
    .add("OnlineBackup", StringType())\
    .add("DeviceProtection", StringType())\
    .add("TechSupport", StringType())\
    .add("StreamingTV", StringType())\
    .add("StreamingMovies", StringType())\
    .add("PaperlessBilling", StringType())\
    .add("PaymentMethod", StringType())\
    .add("MonthlyCharges", IntegerType())\
    .add("TotalCharges", IntegerType())\
    .add("Churn", StringType())



#Read from JSON Kafka messages into a dataframe
dfKafka = spark.read.format("kafka")\
    .option("kafka.bootstrap.servers", srcKafkaBrokers)\
    .option("subscribe", srcKakaTopic)\
    .option("startingOffsets", "earliest")\
    .load()\
    .selectExpr("CAST(value AS STRING) as jsonValue")\
    .rdd.map(lambda row: row["jsonValue"])

dfObj = spark.read.schema(schema).json(dfKafka)



In [ ]:
dataFrom_All = dataFrame_Customer.join(dfObj, "customerID", how="full")

###  Push prepared data to object storage and stop Spark cluster to save resources
###  Note - be sure to change this user_id on the next line to your username (something in the range user1 ... user30)

In [ ]:
user_id = "user29"
file_location = "s3a://data/full_data_csv" + user_id
dataFrom_All.repartition(1).write.mode("overwrite")\
    .option("header", "true")\
    .format("csv").save(file_location)

In [ ]:
spark.stop()